## Quantitative Analysis of Histopathological Images for Autoimmune Diseases Diagnosis

### Brief about the Autoimmune diseases

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load images
img_lupus = mpimg.imread('Medical/Disease Images/Lupus.png')
img_arthritis = mpimg.imread('Medical/Disease Images/Arthritis.png')
img_sclerosis = mpimg.imread('Medical/Disease Images/Sclerosis.png')

# Create a figure with a single row and three columns
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# Display each image
axs[0].imshow(img_lupus)
axs[0].set_title('Lupus')
axs[0].axis('off')

axs[1].imshow(img_arthritis)
axs[1].set_title('Arthritis')
axs[1].axis('off')

axs[2].imshow(img_sclerosis)
axs[2].set_title('Sclerosis')
axs[2].axis('off')

# Adjust layout and display
plt.tight_layout()
plt.show()

### Importing Libraries

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import  GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
import seaborn as sns
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

### Data Collection

In [ ]:
# Data directories
data_dir = 'Medical'
subdirectories = ['Lupus', 'Arthritis', 'Sclerosis']

In [ ]:
fig, axs = plt.subplots(len(subdirectories), 3, figsize=(8, 8))

for i, subdir in enumerate(subdirectories):
    subdir_path = os.path.join(data_dir, subdir)
    for j in range(3):
        filename = os.listdir(subdir_path)[j]
        img_path = os.path.join(subdir_path, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axs[i, j].imshow(img)
        axs[i, j].set_title(subdir)
        axs[i, j].axis('off')

plt.tight_layout()
plt.show()

### Data Preprocessing

In [ ]:
cleaned_images = []

for subdir in subdirectories:
    subdir_path = os.path.join(data_dir, subdir)
    for filename in os.listdir(subdir_path):
        img_path = os.path.join(subdir_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            # Apply Gaussian blur for noise removal
            cleaned_img = cv2.GaussianBlur(img, (5, 5), 0)
            cleaned_images.append((cleaned_img, subdir))
        else:
            print(f"Could not read image: {img_path}")


In [ ]:
# Preprocessing Function
def preprocess_image(img):
    # Resize the image to a desired size (e.g., 224x224)
    resized_img = cv2.resize(img, (224, 224))
    return resized_img

In [ ]:
# Apply Preprocessing to Cleaned Images
preprocessed_images = []
labels = []

In [ ]:
for img, label in cleaned_images:
    preprocessed_img = preprocess_image(img)
    preprocessed_images.append(preprocessed_img)
    labels.append(label)

### Data Visualization

In [ ]:
# Display preprocessed images after Gaussian blur
fig, axs = plt.subplots(len(subdirectories), 3, figsize=(8, 8))

for i, subdir in enumerate(subdirectories):
    subdir_preprocessed_images = [img for img, label in cleaned_images if label == subdir]
    for j in range(3):
        img = subdir_preprocessed_images[j]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for displaying with matplotlib
        axs[i, j].imshow(img)
        axs[i, j].set_title(subdir)
        axs[i, j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Convert lists to numpy arrays
preprocessed_images = np.array(preprocessed_images)
labels = np.array(labels)

In [ ]:
# Encode labels as integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [ ]:
# Convert encoded labels to categorical (one-hot encoding)
categorical_labels = to_categorical(encoded_labels)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_images, categorical_labels, test_size=0.2, random_state=42)

In [ ]:
# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
)

In [ ]:
# Apply data augmentation only to training data
train_generator = datagen.flow(X_train, y_train, batch_size=32)

### Model Evaluation

In [ ]:
# Load VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.summary()

In [ ]:
# Extract features for each preprocessed image using VGG16
def extract_features(data, model):
    features = model.predict(data)
    return features

In [ ]:
# Extract features for training and testing data
train_features = []
for img in X_train:
    img = np.expand_dims(img, axis=0)  # Expand dimensions to match model input
    img = preprocess_input(img)  # Preprocess the image for VGG16
    feature = extract_features(img, base_model)
    train_features.append(feature.flatten())

In [ ]:
test_features = []
for img in X_test:
    img = np.expand_dims(img, axis=0)  # Expand dimensions to match model input
    img = preprocess_input(img)  # Preprocess the image for VGG16
    feature = extract_features(img, base_model)
    test_features.append(feature.flatten())

In [ ]:
# Convert lists to numpy arrays
train_features = np.array(train_features)
test_features = np.array(test_features)

In [ ]:
# Define the model
model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(subdirectories), activation='softmax')  # Output layer with number of classes
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Fit the model
history = model.fit(
    train_features, y_train,
    epochs=20,
    batch_size=30,
    validation_data=(test_features, y_test)
)

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# Classification report and confusion matrix
y_pred = model.predict(test_features)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_true_classes, y_pred_classes, target_names=subdirectories))

In [ ]:
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(conf_matrix, annot=True, xticklabels=subdirectories, yticklabels=subdirectories, cmap="Reds")
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

## Analysis Using Patterns

In [ ]:
# Function to identify patterns
def identify_patterns(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Threshold the image
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Draw contours on the original image
    pattern_img = cv2.drawContours(img.copy(), contours, -1, (0, 255, 0), 3)
    return pattern_img

In [ ]:
# Function to display identified patterns
num_images_per_directory = 5 
# Calculate number of rows and columns for subplots
num_rows = len(subdirectories)
num_cols = num_images_per_directory

fig, axs = plt.subplots(num_rows, num_cols, figsize=(3 * num_cols, 3 * num_rows))

for i, subdir in enumerate(subdirectories):
    subdir_images = preprocessed_images[labels == subdir][:num_images_per_directory]
    for j in range(num_images_per_directory):
        img = subdir_images[j]
        pattern_img = identify_patterns(cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))

        axs[i, j].imshow(pattern_img, cmap='gray')  # Display pattern image in grayscale
        axs[i, j].set_title(f'{subdir}')
        axs[i, j].axis('off')

# Hide empty subplots (not necessary since we ensure the number of images matches the subplots)
for i in range(num_rows):
    for j in range(num_images_per_directory, num_cols):
        axs[i, j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Load the base ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(subdirectories), activation='softmax')(x)

### Model Evaluation

In [ ]:
# Create the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=20, validation_data=(X_test, y_test))

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(history.history['accuracy'], color='blue', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], color='red', label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(history.history['loss'], color='blue', label='Training Loss')
plt.plot(history.history['val_loss'], color='red', label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Prepare to identify correct predictions
correct_images_per_dir = {subdir: [] for subdir in subdirectories}

In [ ]:
# Predict and identify correct images
for subdir in subdirectories:
    subdir_images = [img for img, label in zip(preprocessed_images, labels) if label == subdir][:50]
    
    if len(subdir_images) == 0:
        continue

    subdir_images = np.array(subdir_images)
    predictions = model.predict(subdir_images)
    
    for img, prediction in zip(subdir_images, predictions):
        predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])[0]
        if predicted_label == subdir:
            correct_images_per_dir[subdir].append(img)
        if len(correct_images_per_dir[subdir]) >= 5:
            break

### Final Results

In [ ]:
# Display the first 5 correctly identified images with patterns from each directory
num_images_per_directory = 5

fig, axs = plt.subplots(len(subdirectories), num_images_per_directory, figsize=(3 * num_images_per_directory, 3 * len(subdirectories)))

for i, subdir in enumerate(subdirectories):
    for j, img in enumerate(correct_images_per_dir[subdir]):
        pattern_img = identify_patterns(cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))
        
        axs[i, j].imshow(pattern_img, cmap='gray')  # Display pattern image in grayscale
        axs[i, j].set_title(f'Identified Patterns for {subdir}')
        axs[i, j].axis('off')

    # Hide empty subplots
    for j in range(len(correct_images_per_dir[subdir]), num_images_per_directory):
        axs[i, j].axis('off')

plt.tight_layout()
plt.show()